In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('unified_df.csv')
df = df.drop(columns=[c for c in df.columns if 'Unnamed' in c])
df

,test_classifier_fit_time,test_feature_selector_fit_time,test_acc,test_mcc,test_roc_auc,test_pr_auc,test_mean_inference_time,dataset,n_samples,n_features_org,cv_method,learning_algorithm,filtering_algorithm,n_selected_features,selected_features_names,selected_features_scores
0,0.001552,0.014603,0.930556,0.848620,0.952340,0.958263,0.000714,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,1.0,1881,216.7237
1,0.001559,0.015855,0.958333,0.907651,0.994043,0.989402,0.000695,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,2.0,"1881,2287","216.7237,198.9983"
2,0.001659,0.017396,0.930556,0.848620,0.993191,0.988151,0.000736,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,3.0,"1881,2287,3251","216.7237,198.9983,185.3939"
3,0.001565,0.013124,0.958333,0.907651,0.993191,0.988151,0.000700,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,4.0,"1881,2287,3251,1833","216.7237,198.9983,185.3939,174.7985"
4,0.001660,0.014022,0.944444,0.877447,0.994894,0.990794,0.000736,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,5.0,"1881,2287,3251,1833,4846","216.7237,198.9983,185.3939,174.7985,155.4472"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44761,0.384220,0.149673,0.800000,0.583333,0.841667,0.794932,0.099229,Lung.csv,197,12600,"StratifiedKFold(n_splits=10, random_state=42, ...",RandomForestClassifier,select_fdr_fs,1.0,41325_at,140.4045
44762,0.371591,0.168895,0.800000,0.641624,0.795833,0.767731,0.103855,Lung.csv,197,12600,"StratifiedKFold(n_splits=10, random_state=42, ...",RandomForestClassifier,select_fdr_fs,1.0,41325_at,144.1852
44763,0.408334,0.151420,0.750000,0.566995,0.897222,0.799173,0.097882,Lung.csv,197,12600,"StratifiedKFold(n_splits=10, random_state=42, ...",RandomForestClassifier,select_fdr_fs,1.0,37406_at,128.1498
44764,0.402346,0.037768,0.750000,0.513069,0.819444,0.697591,0.103877,Lung.csv,197,12600,"StratifiedKFold(n_splits=10, random_state=42, ...",RandomForestClassifier,select_fdr_fs,1.0,41325_at,133.4056


In [3]:
df

,Unnamed: 0.2,Unnamed: 0,test_classifier_fit_time,test_feature_selector_fit_time,test_acc,test_mcc,test_roc_auc,test_pr_auc,test_mean_inference_time,dataset,n_samples,n_features_org,cv_method,learning_algorithm,filtering_algorithm,n_selected_features,selected_features_names,selected_features_scores,Unnamed: 0.1
0,0,0,0.001552,0.014603,0.930556,0.848620,0.952340,0.958263,0.000714,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,1.0,1881,216.7237,NaN
1,1,1,0.001559,0.015855,0.958333,0.907651,0.994043,0.989402,0.000695,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,2.0,"1881,2287","216.7237,198.9983",NaN
2,2,2,0.001659,0.017396,0.930556,0.848620,0.993191,0.988151,0.000736,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,3.0,"1881,2287,3251","216.7237,198.9983,185.3939",NaN
3,3,3,0.001565,0.013124,0.958333,0.907651,0.993191,0.988151,0.000700,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,4.0,"1881,2287,3251,1833","216.7237,198.9983,185.3939,174.7985",NaN
4,4,4,0.001660,0.014022,0.944444,0.877447,0.994894,0.990794,0.000736,ALLAML.csv,72,7129,LeaveOneOut(),GaussianNB,select_fdr_fs,5.0,"1881,2287,3251,1833,4846","216.7237,198.9983,185.3939,174.7985,155.4472",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44761,481,481,0.384220,0.149673,0.800000,0.583333,0.841667,0.794932,0.099229,Lung.csv,197,12600,"StratifiedKFold(n_splits=10, random_state=42, ...",RandomForestClassifier,select_fdr_fs,1.0,41325_at,140.4045,NaN
44762,482,482,0.371591,0.168895,0.800000,0.641624,0.795833,0.767731,0.103855,Lung.csv,197,12600,"StratifiedKFold(n_splits=10, random_state=42, ...",RandomForestClassifier,select_fdr_fs,1.0,41325_at,144.1852,NaN
44763,483,483,0.408334,0.151420,0.750000,0.566995,0.897222,0.799173,0.097882,Lung.csv,197,12600,"StratifiedKFold(n_splits=10, random_state=42, ...",RandomForestClassifier,select_fdr_fs,1.0,37406_at,128.1498,NaN
44764,484,484,0.402346,0.037768,0.750000,0.513069,0.819444,0.697591,0.103877,Lung.csv,197,12600,"StratifiedKFold(n_splits=10, random_state=42, ...",RandomForestClassifier,select_fdr_fs,1.0,41325_at,133.4056,NaN
